Importamos las librerías necesarias, vamos a usar requests para hacer la petición a la página web y BeautifulSoup para parsear el HTML. Luego pandas para guardar los datos en un DataFrame.

In [3]:
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd
from sentence_transformers import SentenceTransformer
import numpy as np

c:\Users\Francisco\AppData\Local\Programs\Python\Python312\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:13: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


Definimos funciones utiles para el scraping.
* **Soup_page:** Rcibe como parametro una url y devuelte el objeto soup de la misma.


* **ExtraerInfoLibros:** Recibe como parametro el objeto Soup de la pagina especifica de un libro, extrae la informacion del mismo y devuelve un diccionario con los datos extraidos para luego ser insertados en un DataFrame.

In [63]:
# Función para obtener el objeto Soup de una URL
def Soup_page(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    return soup

# Funcion que extrae la informacion de cada libro 
def ExtraerInfoLibros(book_soup, url):
    table = book_soup.find('table', class_='bibrec')
    rows = table.find_all('tr')

    # Inicializa las variables para almacenar los datos
    author = title = language = release_date = summary = None
    subjects = []

    for row in rows:
        th = row.find('th')
        td = row.find('td')

        if th and td:
            if 'Author' in th.text:
                author = td.text.strip()
            elif 'Title' in th.text:
                title = td.text.strip()
            elif 'Language' in th.text:
                language = td.text.strip()
            elif 'Release Date' in th.text:
                release_date = td.text.strip()
            elif 'Summary' in th.text:
                summary = td.text.strip().replace("(This is an automatically generated summary.)", "").strip()
                # Eliminamos comillas dobles al principio y al final del texto
                summary = summary.strip('"')
            elif 'Subject' in th.text:
                subject_links = td.find_all('a')
                for link in subject_links:
                    subjects.append(link.text.strip())

    # Convertimos la lista de subjects en una cadena separada por comas
    subjects_string = ",".join(subjects)

    # Creamos un diccionario con los datos extraídos
    return {
        'Title': title,
        'Author': author,
        'Summary': summary,
        'Subjects': subjects_string,
        'Release Date': release_date,
        'Language': language,
        'Url': url,
    }

In [64]:
# Obtenemos el objeto Soup de la página donde se encuentran los 1000 libros
url = "https://www.gutenberg.org/browse/scores/top1000.php#books-last1"
soup = Soup_page(url)

# Buscamos la etiqueta <div> con la clase 'page_content' que dentro suyo se encuentra la lista con todos los links a la descripcion de cada libro
book_elements = soup.find_all('div', class_='page_content')
urls = []
base_url = 'https://www.gutenberg.org'

counter = 0
for div in book_elements:
    # Buscamos la etiqueta el <ol> dentro del <div>
    ol = div.find('ol')
    if ol:
        # Recorremos todass las etiquetas <li> dentro del <ol>
        for li in ol.find_all('li'):
            # Buscamos la etiqueta <a> dentro de cada etiqueta <li>
            a_tag = li.find('a')
            if a_tag and 'href' in a_tag.attrs:
                # Obtenemos la URL de la etiqueta <a>
                relative_url = a_tag['href']
                full_url = base_url + relative_url
                urls.append(full_url)

Ahora vamos a recorrer cada link, obtener el Soup de cada pagina y guardar la informacion respectiva de cada libro en un dataframe de Pandas.

In [65]:
# Inicializamos un DataFrame vacío para luego llenarlo con la informacion de los libros
df_libros = pd.DataFrame(columns=['Title', 'Author', 'Summary', 'Subjects', 'Release Date', 'Language', 'Url'])

for url in urls:
    book_soup = soup_page(url)  # Obtén el soup de la página
    book_data = ExtraerInfoLibros(book_soup, url)  # Extrae los datos del libro
    df_libros = pd.concat([df_libros, pd.DataFrame([book_data])], ignore_index=True)  # Agrega la fila al DataFrame

In [66]:
# Guardamos el DataFrame en un archivo CSV
df_libros.to_csv('df_libros.csv', index=False)

# Generación de Embeddings

In [4]:
# Cargar un modelo preentrenado como BERT
model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2')

In [15]:
# Dataset de juegos de mesa
df_juegos_de_mesa = pd.read_csv('bgg_database.csv')
# Dataset de películas
df_peliculas = pd.read_csv('IMDB-Movie-Data.csv')
# Dataset de películas
df_libros = pd.read_csv('df_libros.csv')

In [16]:
df_peliculas['full_text'] = df_peliculas['Genre'] + ' ' + df_peliculas['Description']
df_peliculas['full_text'] = df_peliculas['full_text'].fillna('') # Rellenar los valores nulos con un string vacío

df_libros['full_text'] = df_libros['Subjects'] + ' ' + df_libros['Summary']
df_libros['full_text'] = df_libros['full_text'].fillna('') # Rellenar los valores nulos con un string vacío

df_juegos_de_mesa['full_text'] = df_juegos_de_mesa['categories'] + ' ' + df_juegos_de_mesa['description']
df_juegos_de_mesa['full_text'] = df_juegos_de_mesa['full_text'].fillna('') # Rellenar los valores nulos con un string vacío

In [17]:
# Para películas
peliculas_embeddings = model.encode(df_peliculas['full_text'].tolist())
df_peliculas['embeddings'] = [np.array(embed) for embed in peliculas_embeddings]
print('Peliculas OK!')

# Para libros
libros_embeddings = model.encode(df_libros['full_text'].tolist())
df_libros['embeddings'] = [np.array(embed) for embed in libros_embeddings]
print('Libros OK!')

# Para juegos
juegos_embeddings = model.encode(df_juegos_de_mesa['full_text'].tolist())
df_juegos_de_mesa['embeddings'] = [np.array(embed) for embed in juegos_embeddings]
print('Juegos OK!')

Peliculas OK!
Libros OK!
Juegos OK!


In [ ]:
df_peliculas.to_csv('./datasets_embeddings/df_peliculas_emb.csv', index=False)
df_libros.to_csv('./datasets_embeddings/df_libros_emb.csv', index=False)
df_juegos_de_mesa.to_csv('./datasets_embeddings/df_juegos_de_mesa_emb.csv', index = False)